# Step 1: Login as External Researcher

In [1]:
import syft as sy

datasites = list()
for i in range(3):
    server = sy.orchestra.launch(name="fl-datasite-"+str(i)) # connects to same server
    client = server.login(email="sheldon@caltech.edu", password="changethis")
    datasites.append(client)

SyftInfo: You have launched a development server at http://0.0.0.0:None.It is intended only for local use.

Logged into <fl-datasite-0: High side Datasite> as <sheldon@caltech.edu>


SyftWarning: You are using a default password. Please change the password using `[your_client].account.set_password([new_password])`.

SyftInfo: You have launched a development server at http://0.0.0.0:None.It is intended only for local use.

Logged into <fl-datasite-1: High side Datasite> as <sheldon@caltech.edu>


SyftWarning: You are using a default password. Please change the password using `[your_client].account.set_password([new_password])`.

SyftInfo: You have launched a development server at http://0.0.0.0:None.It is intended only for local use.

Logged into <fl-datasite-2: High side Datasite> as <sheldon@caltech.edu>


SyftWarning: You are using a default password. Please change the password using `[your_client].account.set_password([new_password])`.

# Step 2: Get mock data and test a neural network

In [2]:
mock_images = datasites[0].datasets['MNIST Dataset'].assets['images'].mock[0:100]
mock_labels = datasites[0].datasets['MNIST Dataset'].assets['labels'].mock[0:100]

In [3]:
# third party
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset

# Define the data loader
train_loader = torch.utils.data.DataLoader(
    TensorDataset(torch.tensor(mock_images, dtype=torch.float32), 
                               torch.tensor(mock_labels, dtype=torch.float32)), 
    batch_size=4, 
    shuffle=True
)

# Define the neural network class
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 10)

    def forward(self, x):
        x = torch.log_softmax(self.fc1(x.view(-1, 784)), dim=1)
        return x

# Define the model, optimizer, and loss function
model = MLP()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# Train the model
for epoch in range(10):
    running_loss = 0.0
    for _, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(
        f"Epoch {epoch + 1}, Loss: {(running_loss / len(train_loader)):.4f}",
    )

Epoch 1, Loss: 0.0000
Epoch 2, Loss: 0.0000
Epoch 3, Loss: 0.0000
Epoch 4, Loss: 0.0000
Epoch 5, Loss: 0.0000
Epoch 6, Loss: 0.0000
Epoch 7, Loss: 0.0000
Epoch 8, Loss: 0.0000
Epoch 9, Loss: 0.0000
Epoch 10, Loss: 0.0000


# Step 3: Submit experiment to each datasites

In [4]:
for dasite in datasites:

    mock_images_ptr = dasite.datasets['MNIST Dataset'].assets['images']
    mock_labels_ptr = dasite.datasets['MNIST Dataset'].assets['labels']
    
    
    @sy.syft_function(
        input_policy=sy.ExactMatch(
            mnist_images=mock_images_ptr, 
            mnist_labels=mock_labels_ptr
        ),
        output_policy=sy.SingleExecutionExactOutput(),
    )
    def train(mnist_images, mnist_labels):
        # third party
        import torch
        import torch.nn as nn
        import torch.optim as optim
        from torch.utils.data import TensorDataset

        # Define the data loader
        train_loader = torch.utils.data.DataLoader(
            TensorDataset(torch.tensor(mnist_images, dtype=torch.float32), 
                          torch.tensor(mnist_labels, dtype=torch.float32)), 
            batch_size=4, 
            shuffle=True
        )

        # Define the neural network class
        class MLP(nn.Module):
            def __init__(self):
                super().__init__()
                self.fc1 = nn.Linear(784, 10)

            def forward(self, x):
                x = torch.log_softmax(self.fc1(x.view(-1, 784)), dim=1)
                return x

        # Define the model, optimizer, and loss function
        model = MLP()
        optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
        criterion = nn.CrossEntropyLoss()

        # Train the model
        train_accs = []
        for epoch in range(20):
            running_loss = 0.0
            for _, data in enumerate(train_loader, 0):
                inputs, labels = data
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            print(
                f"Epoch {epoch + 1}, Loss: {(running_loss / len(train_loader)):.4f}"
            )
            # Calculate accuracy on the training set
            train_accs.append((running_loss / len(train_loader)))

        # Get model parameters
        params = model.state_dict()

        # Return training accuracy and model parameters
        return train_accs, params
    new_project = sy.Project(
        name="Training a 3-layer torch neural network on MNIST data",
        description="""Hi, I would like to train my neural network on your MNIST data 
                    (I can download it online too but I just want to use Syft coz it's cool)""",
        members=[dasite],
    )
    new_project.create_code_request(obj=train, client=dasite)
    project = new_project.send()

SyftSuccess: Syft function 'train' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftSuccess: Syft function 'train' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftSuccess: Syft function 'train' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.